In [1]:
from models.yolo import Model
import torch 

In [ ]:
model = Model(opt.cfg or ckpt['model'].yaml, ch=3, nc=nc, anchors=hyp.get('anchors'))

In [1]:
import os
import sys
import time
import re
import csv
import glob
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.data import Dataset


import torchvision
from torchvision import datasets
import torchvision.transforms as transforms
from PIL import Image

# Specify random seed for repeatable results
torch.manual_seed(191009)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [6]:
# Helper functions
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':4.2f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {avg' + self.fmt + '}'
        return fmtstr.format(**self.__dict__)


def accuracy(output, target):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = 1
        batch_size = target.size(0)

        _, pred = output.topk(1, 1, True, True)
        pred = pred.t()

        xx = torch.logical_xor(pred.view(pred.shape[1]), target)
        fn = sum(torch.logical_and(xx, target))
        fp = sum(xx) - fn

        correct = pred.eq(target.view(1, -1).expand_as(pred))

        correct_k = correct[:1].reshape(-1).float().sum(0, keepdim=True)
        return correct_k.mul_(100.0 / batch_size), fn * 100.0 / 16, fp * 100.0 / 16


def evaluate(model, criterion, data_loader):
    model.eval()
    top1 = AverageMeter('Eval_Acc', ':4.2f')
    fpr = AverageMeter('Eval_FPR', ':4.2f')
    fnr = AverageMeter('EVAL_FPR', ':4.2f')
    cnt = 0
    with torch.no_grad():
        for image, target in data_loader:
            image, target = image.to(device), target.to(device)
            output = model(image)
            loss = criterion(output, target)
            cnt += 1
            acc1, fn, fp = accuracy(output, target)
#             print('.', end = '')
            top1.update(acc1[0], image.size(0))
            fpr.update(fp)
            fnr.update(fn)

    return top1, fpr, fnr

def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [7]:
# Custom Dataset Class to load from self-defined dir-structure
class ZeroOneSet(Dataset):
    def __init__(self, data_dir, ext='png'):
        zeros = list(glob.glob(f'{data_dir}/0/*.{ext}'))
        ones = list(glob.glob(f'{data_dir}/1/*.{ext}'))
        self.data = zeros + ones
        self.labels = {}
        for i in zeros:
            self.labels[i.split('/')[-1]] = 0
        for i in ones:
            self.labels[i.split('/')[-1]] = 1

        self.classes = 2
        self.transform = transforms.Compose(
                            [torchvision.transforms.Grayscale(num_output_channels=3),
                             transforms.ToTensor(),
                             transforms.Normalize((0.5,), (0.5,))])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        path = self.data[idx]
        file = os.path.basename(path)
        img = Image.open(path)
        label = int(self.labels[file] > 0)

        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [15]:
# from google.colab import drive
# drive.mount('/content/drive')
!tar -xf ./10000LANC.tar
# !ls
# !rm -rf forImgClassifyCompressed/
# !ls


In [16]:
data_path = './forImgClassifyCompressed'
# data_path = '/Users/xinye/Downloads/trailcamDataset'

BATCH = 16

# read whole set
IMG_PATH = os.path.join(data_path, 'img')
dataset = ZeroOneSet(IMG_PATH)

# split train val test set
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
trainset, testset = torch.utils.data.random_split(dataset, [train_size, test_size])

trainloader = DataLoader(trainset, batch_size=BATCH, shuffle=True)
testloader = DataLoader(testset, batch_size=BATCH, shuffle=True)

In [17]:
model = torchvision.models.mobilenet_v2(num_classes = 2).to(device)

data_loader, data_loader_test = (trainloader, testloader)
optimizer = torch.optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [18]:
'''TRAIN'''
def train(model: nn.Module, dataloader: DataLoader):
    for epoch in range(20):  # loop over the dataset multiple times
        model.train()
        running_loss = AverageMeter('loss')
        acc = AverageMeter('train_acc')
        fpr = AverageMeter('train_fpr')
        fnr = AverageMeter('train_fnr')
        for i, data in enumerate(dataloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)


            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics

            acc1, fn, fp = accuracy(outputs, labels)
            fpr.update(fp)
            fnr.update(fn)

            running_loss.update(loss.item(), outputs.shape[0])
            acc.update(acc1[0], outputs.shape[0])
            if i % 100 == 0:    # print every 100 mini-batches
                print('[%d, %4d]' % (epoch + 1, i + 1), running_loss, acc, fpr, fnr)
        model.eval()
        topEval, fprEval, fnrEval = evaluate(model, criterion, data_loader_test)
        print('[%d, %4d]' % (epoch + 1, i + 1), topEval, fprEval, fnrEval)

    print('Finished Training')

train(model, data_loader)

[1,    1] loss 0.64 train_acc 68.75 train_fpr 31.25 train_fnr 0.00
[1,  101] loss 0.81 train_acc 52.29 train_fpr 24.01 train_fnr 23.70
[1,  201] loss 0.75 train_acc 56.69 train_fpr 21.92 train_fnr 21.39
[1,  274] Eval_Acc 56.95 Eval_FPR 1.54 EVAL_FPR 41.12
[2,    1] loss 0.79 train_acc 50.00 train_fpr 25.00 train_fnr 25.00
[2,  101] loss 0.73 train_acc 62.93 train_fpr 18.44 train_fnr 18.63
[2,  201] loss 0.68 train_acc 65.73 train_fpr 16.82 train_fnr 17.44
[2,  274] Eval_Acc 62.98 Eval_FPR 1.54 EVAL_FPR 35.14
[3,    1] loss 1.02 train_acc 56.25 train_fpr 6.25 train_fnr 37.50
[3,  101] loss 0.59 train_acc 71.41 train_fpr 13.06 train_fnr 15.53
[3,  201] loss 0.57 train_acc 72.23 train_fpr 12.78 train_fnr 14.99
[3,  274] Eval_Acc 73.95 Eval_FPR 11.23 EVAL_FPR 14.58
[4,    1] loss 0.33 train_acc 87.50 train_fpr 6.25 train_fnr 6.25
[4,  101] loss 0.59 train_acc 75.80 train_fpr 11.26 train_fnr 12.93
[4,  201] loss 0.60 train_acc 75.19 train_fpr 11.41 train_fnr 13.40
[4,  274] Eval_Acc 78.52 

In [19]:
num_eval_batches = 1000

print("Size of baseline model")
print_size_of_model(model)

top1, fpr, fnr = evaluate(model, criterion, data_loader_test)
print('accuracy: %6.2f' % top1.avg)
print('FPR: %6.2f' % fpr.avg)
print('FNR: %6.2f' % fnr.avg)

Size of baseline model
Size (MB): 9.128025
accuracy:  80.62
FPR:  16.76
FNR:   2.45


In [20]:
savePath = os.path.join(os.getcwd(), 'savedModelForTable')
fileName = 'MobileNetV2_EuroCity.pt'
if not os.path.exists(savePath):
    os.mkdir(savePath)
torch.save(model.state_dict(), os.path.join(savePath, fileName))

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Conv(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, stride=1, padding=0):
        super(Conv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

class TinyYOLOv5(nn.Module):
    def __init__(self, num_classes=80):
        super(TinyYOLOv5, self).__init__()

        self.conv1 = Conv(3, 16, kernel_size=3, stride=1, padding=1)
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = Conv(16, 32, kernel_size=3, stride=1, padding=1)
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = Conv(32, 64, kernel_size=3, stride=1, padding=1)
        self.maxpool3 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv4 = Conv(64, 128, kernel_size=3, stride=1, padding=1)
        self.maxpool4 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv5 = Conv(128, 256, kernel_size=3, stride=1, padding=1)
        self.maxpool5 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv6 = Conv(256, 512, kernel_size=3, stride=1, padding=1)
        self.maxpool6 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv7 = Conv(512, 1024, kernel_size=3, stride=1, padding=1)
        self.conv8 = Conv(1024, 256, kernel_size=1, stride=1)
        self.conv9 = Conv(256, 512, kernel_size=3, stride=1, padding=1)
        self.conv10 = nn.Conv2d(512, num_classes, kernel_size=1, stride=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.conv4(x)
        x = self.maxpool4(x)
        x = self.conv5(x)
        x = self.maxpool5(x)
        x = self.conv6(x)
#         x = self.maxpool6(x)
        x = self.conv7(x)
        x = self.conv8(x)
        x = self.conv9(x)
        x = self.conv10(x)
        x = F.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        return x

# Create an instance of the TinyYOLOv5 model
model = TinyYOLOv5()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from tqdm import tqdm

# Set random seed for reproducibility
torch.manual_seed(42)

# Define the TinyYOLOv5 model and its optimizer
model = TinyYOLOv5()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Load CIFAR10 dataset for training
train_dataset = CIFAR10(root="./data", train=True, transform=ToTensor(), download=False)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    # Set the model to training mode
    model.train()
    train_loss = 0.0
    train_correct = 0

    # Iterate over the training batches
    for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # Compute loss
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        train_loss += loss.item() * images.size(0)
        train_correct += (predicted == labels).sum().item()

    # Compute average training loss and accuracy
    train_loss /= len(train_dataset)
    train_accuracy = 100.0 * train_correct / len(train_dataset)

    # Print training metrics
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")


In [ ]:

for epoch in range(num_epochs):
    # Set the model to training mode
    ef.train()
    train_loss = 0.0
    train_correct = 0

    # Iterate over the training batches
    for images, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = ef(images)
        _, predicted = torch.max(outputs, 1)

        # Compute loss
        loss = criterion(outputs, labels)
#         print(predicted)
#         print(labels)
#         print()
#         print()

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Track training loss and accuracy
        train_loss += loss.item() * images.size(0)
        train_correct += (predicted == labels).sum().item()

    # Compute average training loss and accuracy
    train_loss /= len(train_dataset)
    train_accuracy = 100.0 * train_correct / len(train_dataset)

    # Print training metrics
    print(f"Epoch {epoch+1}/{num_epochs}:")
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.2f}%")


In [ ]:
ef = torchvision.models.efficientnet_b0(num_classes=10)

In [ ]:
model

In [ ]:
model.conv10

In [ ]:
from torchsummary import summary
summary(model,(3,32,32))

In [ ]:
summary(ef,(3,32,32))


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms


# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define transforms
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

# Initialize EfficientNet-B0 model
model = torchvision.models.efficientnet_b0(pretrained=False, num_classes=10).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # Print epoch statistics
    train_loss = running_loss / len(trainloader)
    train_acc = 100. * correct / total
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch+1, num_epochs, train_loss, train_acc))

    # Test the model
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss /= len(testloader)
    test_acc = 100. * correct / total
    print('Test Loss: {:.4f}, Test Accuracy: {:.2f}%'
          .format(test_loss, test_acc))

print('Finished Training')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from tqdm import tqdm


# Set random seed for reproducibility
torch.manual_seed(42)


# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=ToTensor())
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=ToTensor())
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

# Initialize EfficientNet-B0 model
model = torchvision.models.efficientnet_b0(pretrained=False, num_classes=10).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for i, (inputs, labels) in enumerate(trainloader):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    # Print epoch statistics
    train_loss = running_loss / len(trainloader)
    train_acc = 100. * correct / total
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
          .format(epoch+1, num_epochs, train_loss, train_acc))

    # Test the model
    model.eval()
    test_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    test_loss /= len(testloader)
    test_acc = 100. * correct / total
    print('Test Loss: {:.4f}, Test Accuracy: {:.2f}%'
          .format(test_loss, test_acc))

print('Finished Training')


In [ ]:
torch.device("cuda" if torch.cuda.is_available() else "cpu")
